## How can I update layers in my web map to use HTTPS?

As the owner or administrator of a web map, you can update all layers in the web map to use HTTPS.  This can be done either by the GUI on ArcGIS Online from the Settings tab of the map's item page or through the ArcGIS API for Python. It is recommended that you access all layers using HTTPS, which encrypts the information in the layer when it's transmitted over the Internet.

When you update the layers in your map to use HTTPS, ArcGIS Online inspects each layer in the map to determine whether it needs to be updated to HTTPS. If any layers are found to use HTTP, ArcGIS Online attempts to make a request to the same layer using HTTPS and updates the map and any associated layer items you own or administer. If a layer does not support HTTPS, you are notified of this fact and the layer is not updated in the map. If you are notified that the layer does not support HTTPS, contact the owner of the layer, who can either configure the layer to support HTTPS or provide an alternative resource.

When you attempt to update layers in your map to use HTTPS, any custom symbols that use HTTP are automatically updated to use HTTPS if the source of the custom symbols supports it. Other layer customizations, such as images embedded in pop-ups, are not updated to HTTPS. To update these references to use HTTPS, open the layer (or map) in Map Viewer and update the references there.

In [29]:
from arcgis.gis import GIS
from arcgis.gis import Item

### Connect to the GIS

In [ ]:
gis = GIS(url="https://python.playground.esri.com/portal",
          username="arcgis_python")

### Copy the Item for the Demo

In [37]:
wm = gis.content.get("722f8121d63242b98366e012e4431e13")
wm = wm.copy(title="%s - ALL HTTPS" % wm.title)

In [38]:
wm

<Item title:"HTTPtoHTTPSExample - ALL HTTPS" type:Web Map owner:andrew>

### Updating the HTTP Scheme to HTTPS Scheme

The code below walks the operational layers and checks if the service can be updated to HTTPS.

In [33]:
import json
from arcgis.features import FeatureLayer
from arcgis.raster import ImageryLayer
from arcgis.mapping import MapImageLayer, VectorTileLayer
def update_http_to_https(wm):
     """
     Updates an Item's JSON by Changing All HTTP Scheme to HTTPS.
     
     ============   =====================================
     **Inputs**     **Description**
     ------------   -------------------------------------
     wm             Required Item. Must be a Web Map Item.
     ============   =====================================
     
     :returns: boolean
     """

     if wm.type == 'Web Map':
          data = wm.get_data()
          if 'operationalLayers' in data:
               for lyr in data['operationalLayers']:
                    if 'url' in lyr:
                         url = lyr['url']
                         if url.lower().find("http://") > -1:
                              try:
                                   if lyr['layerType'] == 'ArcGISFeatureLayer':
                                        url = url.replace("http://", "https://")
                                        fl = FeatureLayer(url, gis=gis)
                                        fl.properties
                                   elif lyr['layerType'] == 'ArcGISTiledMapServiceLayer':
                                        url = url.replace("http://", "https://")
                                        fl = MapImageLayer(url, gis=gis)
                                        fl.properties
                                   elif lyr['layerType'] == 'ArcGISImageServiceLayer':
                                        url = url.replace("http://", "https://")
                                        fl = ImageryLayer(url, gis=gis)
                                        fl.properties
                                   lyr['url'] = url
                              except:
                                   print("HTTPS is not supported on %s" % lyr['url'])
                              else:
                                   print("Updated service to: %s" % lyr['url'])
                    else:
                         print("Cannot update layer: %s" % lyr)

               wm.update({"text" : json.dumps(data)})
     return True

In [34]:
update_http_to_https(wm)

Cannot update layer: {'layerType': 'VectorTileLayer', 'type': 'VectorTileLayer', 'opacity': 1, 'itemId': 'aacf0898e01144f7b3f981ab18fc3626', 'title': 'World_earthquakes_2010', 'visibility': True, 'id': 'VectorTile_7217', 'styleUrl': 'http://python.playground.esri.com/server/rest/services/Hosted/World_earthquakes_2010/VectorTileServer/resources/styles/root.json'}
Updated service to: https://python.playground.esri.com/server/rest/services/Hosted/earthquakes/FeatureServer/0


True

In [35]:
from arcgis.mapping import WebMap
WebMap(wm)

We now have a copy of a web map that is HTTPS verse the default version that has HTTP service URLs.